## 策略分析与回测-近远月价差的布林带策略

基于近远月价差的在某一范围内上下波动，出现均值回归，具有协整性的特点。这种方法的核心思想是，价差会在其均值附近波动。当价差显著偏离其均值（例如，超过若干倍标准差）时，预期它会向均值回归，从而产生套利机会，具体来说，步骤如下：

1. 选择一个回顾窗口：例如，过去N个数据（N分钟或N天）。
2. 计算价差的滚动均值：回顾期内价差的平均值。
3. 计算价差的滚动标准差：回顾期内价差的标准差。
4. 设定开仓阈值：做空价差（例如，卖出当月合约，买入下季合约，当“当月-下季”价差过高时）： 开仓阈值 = 滚动均值 + N_open * 滚动标准差
5. 设定平仓阈值：当价差回归时平仓： 平仓阈值 = 滚动均值 + N_close * 滚动标准差 （通常 0 <= N_close < N_open，例如，价差回到均值附近或均值加上一个较小的标准差倍数时平仓）

在这个简单策略中，有许多可以讨论的环节：
1. 交易信号回顾窗口的长度
2. 开仓阈值
3. 平仓阈值
对于同一个标的来说，不同到期月的交易对组合，开仓阈值和平仓阈值的设置也是不同的。

- 数据选择：沪深300股指期货当月数据和下季月分钟数据收盘价
- 回测平台：JoinQuant
- 参数设置：初始投资为100万元，保证金比率为12%，交易费率万分之0.23，平今万分之2.3，价格滑点为2，每次固定开仓量为5张。

In [ ]:
# JoinQuant 代码，复制到聚宽回测环境中实现


# 导入函数库
from jqdata import *
import numpy as np
import pandas as pd
from collections import deque
import datetime # 导入datetime模块

## 初始化函数，设定基准等等
def initialize(context):
    set_benchmark('000300.XSHG')
    set_option('use_real_price', True)
    log.info('初始函数开始运行且全局只运行一次')

    set_subportfolios([SubPortfolioConfig(cash=context.portfolio.starting_cash, type='index_futures')])
    set_order_cost(OrderCost(open_commission=0.000023, close_commission=0.000023,close_today_commission=0.000023), type='index_futures')
    set_option('futures_margin_rate', 0.15)
    set_slippage(StepRelatedSlippage(2))

    # --- 移仓换月参数 ---
    g.days_before_expiry_to_roll = 3 # 在当月合约到期前几天开始考虑移仓

    # --- 动态阈值参数 ---
    g.lookback_window = 120
    g.N_open = 3.5
    g.N_close = 0.3
    # --- 其他全局变量 ---
    g.IF_current_month = None
    g.IF_next_quarter = None
    g.trade_volume = 5
    g.historical_spreads = deque(maxlen=g.lookback_window)
    g.dynamic_open_threshold = None
    g.dynamic_close_threshold = None
    g.spread_mean = None
    g.spread_std = None
    g.is_roll_pending_execution = False # 标记是否有移仓平仓操作需要在开盘后执行

    # run_daily 函数
    run_daily(before_market_open, time='before_open', reference_security='IF8888.CCFX') # 修改为 before_open
    run_daily(trade_every_minute, time="every_bar", reference_security='IF8888.CCFX')
    run_daily(after_market_close, time='15:30', reference_security='IF8888.CCFX')

## 开盘前运行函数 (每日更新合约代码并检查是否需要移仓)
def before_market_open(context):
    log.info(f'函数运行时间(before_market_open): {str(context.current_dt.time())}')
    
    # 1. 获取最新的目标合约
    try:
        latest_target_current_month = get_future_contracts('IF')[0]
        latest_target_next_quarter = get_future_contracts('IF')[2]
    except Exception as e:
        log.error(f"开盘前获取最新目标合约代码失败: {e}")
        return # 如果获取失败，则不进行后续操作，保持现有合约（如果有）

    # 2. 检查是否是策略首次运行或合约未初始化
    if g.IF_current_month is None or g.IF_next_quarter is None:
        log.info("首次运行或合约未初始化，设置目标合约。")
        g.IF_current_month = latest_target_current_month
        g.IF_next_quarter = latest_target_next_quarter
        g.historical_spreads.clear() # 清空历史价差
        g.is_roll_pending_execution = False
        log.info(f"初始化当日操作合约：当月 {g.IF_current_month}, 下季 {g.IF_next_quarter}")
        return

    # 3. 检查是否需要移仓
    roll_needed = False
    reason_for_roll = ""

    # 条件A: 当前跟踪的当月合约不再是 get_future_contracts('IF')[0] 返回的合约
    if g.IF_current_month != latest_target_current_month:
        roll_needed = True
        reason_for_roll = f"当月合约定义发生变化 (原: {g.IF_current_month}, 新: {latest_target_current_month})"
    
    # 条件B: 当前跟踪的当月合约临近到期
    if not roll_needed: # 只有当条件A不满足时才检查条件B，避免重复判断
        current_month_end_date = get_CCFX_end_date(g.IF_current_month)
        days_to_expiry = (current_month_end_date - context.current_dt.date()).days
        if days_to_expiry <= g.days_before_expiry_to_roll:
            roll_needed = True
            reason_for_roll = f"当月合约 {g.IF_current_month} 临近到期 ({days_to_expiry} 天)，最后交易日: {current_month_end_date}"

    # 4. 执行移仓逻辑
    if roll_needed:
        log.info(f"需要移仓换月！原因: {reason_for_roll}")
        
        # 检查是否有旧合约的持仓需要平掉
        old_current_month_short_pos = context.portfolio.short_positions.get(g.IF_current_month)
        old_next_quarter_long_pos = context.portfolio.long_positions.get(g.IF_next_quarter)

        if (old_current_month_short_pos and old_current_month_short_pos.total_amount > 0) or \
           (old_next_quarter_long_pos and old_next_quarter_long_pos.total_amount > 0):
            log.info(f"持有旧合约头寸，标记在开盘后执行平仓。旧当月: {g.IF_current_month}, 旧下季: {g.IF_next_quarter}")
            g.is_roll_pending_execution = True # 设置标记，让 trade_every_minute 在开盘初执行平仓
        else:
            log.info("旧合约无持仓，无需执行平仓。")
            g.is_roll_pending_execution = False

        # 更新为新的目标合约
        g.IF_current_month = latest_target_current_month
        g.IF_next_quarter = latest_target_next_quarter
        g.historical_spreads.clear() # 清空历史价差为新合约组合做准备
        g.dynamic_open_threshold = None # 重置动态阈值
        g.dynamic_close_threshold = None
        log.info(f"移仓换月：更新目标合约为 当月 {g.IF_current_month}, 下季 {g.IF_next_quarter}")
    else:
        log.info(f"当日无需移仓换月。当前操作合约：当月 {g.IF_current_month}, 下季 {g.IF_next_quarter}")
        g.is_roll_pending_execution = False


## 分钟级交易函数
def trade_every_minute(context):
    # 在开盘后的第一个bar执行待处理的移仓平仓操作
    if g.is_roll_pending_execution:
        # 需要找到移仓前记录的合约代码来平仓，因为g.IF_current_month等已被更新为新合约
        # 这部分逻辑需要在before_market_open中更妥善地传递旧合约代码，或者在initialize中保存上一日的合约
        # 为简化，我们假设before_market_open在更新g.IF_current_month前，已将其旧值用于决策，
        # 此处平仓逻辑需要访问的是“被替换掉的”旧合约。
        # 一个更稳健的做法是before_market_open中不直接平仓，而是将要平仓的旧合约列表存入g中，
        # 然后在这里迭代平仓。

        # 移仓的平仓应该在 before_market_open 中记录好需要平仓的旧合约代码
        # 然后在 trade_every_minute 的开头，检查这个列表，对列表中的旧合约进行平仓。
        # 为避免代码过于复杂，这里暂时先打印一个移仓提示，实际平仓需精确化。
        # 或者，更简单的方式是，如果before_market_open检测到需要移仓并且有仓位，
        # 它应该记录下这些旧的合约代码，然后在这里，如果g.is_roll_pending_execution为True，
        # 我们就清掉所有 IF 的多头和空头仓位。这是一个比较粗暴但能确保清掉旧仓位的方法。

        log.info("检测到移仓换月后的首个交易bar，尝试清理所有IF合约的遗留头寸...")
        portfolio_updated = False
        for sec_code, pos in list(context.portfolio.long_positions.items()): # 使用list创建副本迭代
            if pos.security.startswith('IF') and pos.total_amount > 0:
                log.info(f"移仓：平掉多头 {sec_code} 数量 {pos.total_amount}")
                order_target(sec_code, 0, side='long')
                portfolio_updated = True
        for sec_code, pos in list(context.portfolio.short_positions.items()): # 使用list创建副本迭代
            if pos.security.startswith('IF') and pos.total_amount > 0:
                log.info(f"移仓：平掉空头 {sec_code} 数量 {pos.total_amount}")
                order_target(sec_code, 0, side='short')
                portfolio_updated = True
        
        if portfolio_updated:
            g.historical_spreads.clear() # 清理价差数据，因为合约组合已变
            g.dynamic_open_threshold = None
            g.dynamic_close_threshold = None
            log.info("遗留头寸清理完毕，重置历史价差和动态阈值。")

        g.is_roll_pending_execution = False # 完成执行后重置标记
        return # 清理完遗留仓位后，本bar不做新的开仓判断，等待下一bar数据


    if not g.IF_current_month or not g.IF_next_quarter:
        return

    current_data = get_current_data()
    try:
        IF_current_month_price = current_data[g.IF_current_month].last_price
        IF_next_quarter_price = current_data[g.IF_next_quarter].last_price

        if pd.isna(IF_current_month_price) or pd.isna(IF_next_quarter_price):
            return
    except Exception as e:
        return

    current_price_spread = IF_current_month_price - IF_next_quarter_price
    g.historical_spreads.append(current_price_spread)

    if len(g.historical_spreads) >= g.lookback_window:
        spread_series = pd.Series(list(g.historical_spreads))
        g.spread_mean = spread_series.mean()
        g.spread_std = spread_series.std()

        if g.spread_std > 0.01: # 增加一个小的阈值避免std过小
            g.dynamic_open_threshold = g.spread_mean + g.N_open * g.spread_std
            g.dynamic_close_threshold = g.spread_mean + g.N_close * g.spread_std
            if g.dynamic_open_threshold <= g.dynamic_close_threshold:
                 # 如果计算出的开仓阈值不合理（例如小于等于平仓阈值），则暂时不更新，使用上一次的有效阈值或等待
                # log.warn(f"动态开仓阈值 {g.dynamic_open_threshold:.2f} 不大于平仓阈值 {g.dynamic_close_threshold:.2f}，本轮不更新开仓阈值。")
                if g.dynamic_open_threshold is not None and g.dynamic_close_threshold is not None : #确保不是None
                    g.dynamic_open_threshold = g.dynamic_close_threshold + 0.1 # 保证开仓阈值至少比平仓阈值大一点
                else: # 如果初始就是None或者不合理，则无法交易
                    return 
        else:
            g.dynamic_open_threshold = None 
            g.dynamic_close_threshold = None
            return 
    else:
        return 

    if g.dynamic_open_threshold is None or g.dynamic_close_threshold is None:
        return

    end_date_current_month = get_CCFX_end_date(g.IF_current_month)

    # --- 开仓逻辑 ---
    if current_price_spread > g.dynamic_open_threshold:
        if context.current_dt.date() == end_date_current_month:
            pass
        else:
            has_current_month_short = g.IF_current_month in context.portfolio.short_positions and context.portfolio.short_positions[g.IF_current_month].total_amount > 0
            has_next_quarter_long = g.IF_next_quarter in context.portfolio.long_positions and context.portfolio.long_positions[g.IF_next_quarter].total_amount > 0
            
            if not has_current_month_short and not has_next_quarter_long:
                log.info(f'开仓---价差: {current_price_spread:.2f} > 动态开仓阈值: {g.dynamic_open_threshold:.2f}')
                log.info(f'操作：做空 {g.trade_volume} 手 {g.IF_current_month}, 做多 {g.trade_volume} 手 {g.IF_next_quarter}')
                order(g.IF_current_month, g.trade_volume, side='short')
                order(g.IF_next_quarter, g.trade_volume, side='long')

    # --- 平仓逻辑 ---
    current_month_position = context.portfolio.short_positions.get(g.IF_current_month)
    next_quarter_position = context.portfolio.long_positions.get(g.IF_next_quarter)

    if current_month_position and current_month_position.total_amount > 0 and \
       next_quarter_position and next_quarter_position.total_amount > 0:
        # 条件1: 价差回归到平仓阈值以下
        condition_close_threshold = current_price_spread < g.dynamic_close_threshold
        # 条件2: 当月合约交割日临近收盘强制平仓
        condition_expiry_force_close = (context.current_dt.date() == end_date_current_month and \
                                        context.current_dt.time() >= datetime.time(14,50)) # 使用 datetime.time

        if condition_close_threshold:
            log.info(f'平仓(阈值回归)---价差: {current_price_spread:.2f} < 动态平仓阈值: {g.dynamic_close_threshold:.2f}')
            order_target(g.IF_current_month, 0, side='short')
            order_target(g.IF_next_quarter, 0, side='long')
        elif condition_expiry_force_close:
            log.info(f"平仓(交割日强制)---当月合约 {g.IF_current_month}。价差: {current_price_spread:.2f}")
            order_target(g.IF_current_month, 0, side='short')
            order_target(g.IF_next_quarter, 0, side='long')


## 收盘后运行函数
def after_market_close(context):
    log.info(f'函数运行时间(after_market_close): {str(context.current_dt.time())}')
    trades = get_trades()
    for _trade in trades.values():
        log.info(f'成交记录: {_trade}')
    log.info(f'当日结束持仓: 多头 {context.portfolio.long_positions}, 空头 {context.portfolio.short_positions}')
    log.info('一天结束')
    log.info('##############################################################')

########################## 获取期货合约信息，请保留 #################################
def get_CCFX_end_date(future_code):
    return get_security_info(future_code).end_date


## 回测结果-IF00-IF03

### 1.开仓阈值的影响
1.1 控制回顾期为120分钟，回落平仓阈值为0.3倍标准差，调整开仓阈值为2、3、4倍标准差

当开仓阈值为2倍标准差（或小于2倍标准差）时，由于开仓信号过于频繁，导致交易次数过多，考虑到高昂的平今成本，回测结果不佳。

![IF开仓阈值为2倍标准差](pic/120min-std2-0.3.png)

当开仓阈值为3倍标准差时，回测结果如下

![IF开仓阈值为3倍标准差](pic/120min-std3-0.3.png)

当开仓阈值为4倍标准差时，回测结果如下

![IF开仓阈值为4倍标准差](pic/120min-std4-0.3.png)

当开仓阈值为5倍标准差时，回测结果如下

![IF开仓阈值为5倍标准差](pic/120min-std5-0.3.png)

综上，综合考虑shape，收益率，开仓阈值在4倍标准差左右。

### 2.平仓阈值的影响


